In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import time
import timeit
from timeit import default_timer as timer
from sklearn.model_selection import StratifiedKFold

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import ExtraTreesClassifier

In [3]:
from sklearn.neighbors import KNeighborsClassifier

In [4]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [5]:
train_df = pd.read_csv("TrainDataFrame.csv")
test_df = pd.read_csv("TestDataFrame.csv")

## Classifiers

In [6]:
dsc = DecisionTreeClassifier()
mnb = MultinomialNB()
rf = RandomForestClassifier()
lr = LogisticRegression()
ex = ExtraTreesClassifier()

In [7]:
#svm1 = SVC() 

In [8]:
x_train = train_df.iloc[:,1:-1]
y_train = train_df.iloc[:,-1]
x_test = test_df.iloc[:,1:-1]
y_test = test_df.iloc[:,-1]

In [9]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape #features = 6k, sq. root of 6k = 77.45

((15997, 6000), (15997,), (4000, 6000), (4000,))

## fit

In [10]:
start = time.clock()
dsc.fit(x_train, y_train)
print (time.clock() - start)

55.51689875170293


In [11]:
start = time.clock()
mnb.fit(x_train, y_train)
print (time.clock() - start)

2.2645807813365977


In [12]:
start = time.clock()
rf.fit(x_train, y_train)
print (time.clock() - start)

11.244066247595207


In [13]:
start = time.clock()
lr.fit(x_train, y_train)
print (time.clock() - start)

40.75867977555656


In [14]:
start = time.clock()
ex.fit(x_train, y_train)
print (time.clock() - start)

17.25441930100996


In [15]:
#svm1.fit(x_train, y_train)

## Predict

In [16]:
y_pred = mnb.predict(x_test)
#confusion_matrix(y_test, y_pred)
#classification_report(y_test, pred)

## Score

In [17]:
dsc.score(x_test, y_test)

0.5495

In [18]:
mnb.score(x_test, y_test)

0.74725

In [19]:
rf.score(x_test, y_test)

0.6225

In [20]:
lr.score(x_test, y_test)

0.7315

In [21]:
ex.score(x_test, y_test)

0.66525

## other

In [22]:
# knn = KNeighborsClassifier(n_neighbors=5)
# knn.fit(x_train, y_train)

In [23]:
#knn.score(x_test, y_test) #score = 0.3895

In [24]:
# from sklearn.pipeline import Pipeline

In [25]:
# text_clf = Pipeline([('clf', MultinomialNB())
#                     ])

In [26]:
# text_clf = text_clf.fit(x_train, y_train)

In [27]:
# predicted = text_clf.predict(x_test)
# np.mean(predicted == y_test)

In [28]:
from sklearn.linear_model import SGDClassifier

In [29]:
#sdg = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42) 
sdg = SGDClassifier()

In [30]:
start = time.clock()
sdg.fit(x_train, y_train)
print (time.clock() - start)

C:\Users\Percy\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


27.104581412802446


In [31]:
sdg.score(x_test, y_test)

0.693

## STACKING

In [32]:
def Stacking(model, train, y, test, n_fold):
    folds = StratifiedKFold(n_splits=n_fold, random_state=1)
    test_pred = np.empty((test.shape[0],1),float)
    train_pred = np.empty((0,1),float)
    for train_indices, val_indices in folds.split(train,y.values):
        x_train,x_val = train.iloc[train_indices],train.iloc[val_indices]
        y_train,y_val = y.iloc[train_indices],y.iloc[val_indices]

        model.fit(X=x_train,y=y_train)
        train_pred = np.append(train_pred,model.predict(x_val))
        test_pred = np.append(test_pred,model.predict(test))
    return test_pred.reshape(-1,1),train_pred


In [33]:
test_pred1 ,train_pred1 = Stacking(model=dsc, n_fold=10, train=x_train, test=x_test, y=y_train)

train_pred1=pd.DataFrame(train_pred1)
test_pred1=pd.DataFrame(test_pred1)

In [34]:
test_pred2 ,train_pred2=Stacking(model=rf, n_fold=10, train=x_train, test=x_test, y=y_train)

train_pred2=pd.DataFrame(train_pred2)
test_pred2=pd.DataFrame(test_pred2)

In [35]:
df = pd.concat([train_pred1, train_pred2], axis=1)
df_test = pd.concat([test_pred1, test_pred2], axis=1)

mnb.fit(df,y_train)
mnb.score(df_test, y_test)

ValueError: could not convert string to float: 'alt.atheism'